In [1]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn.functional as F
from torch import nn, optim

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import pandas as pd
import numpy as np

import random

In [2]:
EPOCH = 20
BATCH_SIZE = 64
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("Using Device:", DEVICE)

trainset = datasets.MNIST(
    root = './.data/',
    train = True,
    download = True,
    transform = transforms.ToTensor()
)

train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2
)

test_loader = datasets.MNIST(
    root = './.data/',
    train = False,
    download = True,
    transform = transforms.ToTensor()
)

Using Device: cuda


In [3]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 4)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.Sigmoid(),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28*28),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        classified = self.classifier(encoded)
        decoded = self.decoder(encoded)
        
        return encoded, classified, decoded

In [4]:
autoencoder = AE().to(DEVICE)
ae_optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001) # weight_decay=0.0000001
ae_criterion = nn.MSELoss()
classify_optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.01)
classify_criterion = nn.CrossEntropyLoss()
ae_loss_history= []
ae_val_loss_history = []
view_data = trainset.data[10:20].view(-1, 28*28)
view_data = view_data.type(torch.FloatTensor)/255

In [5]:
loss_last_history=[]

In [6]:
def ae_train(autoencoder, train_loader):
    autoencoder.train()
    for step, (x, label) in enumerate(train_loader):
        x = x.view(-1, 28*28).to(DEVICE)
        y = x.view(-1, 28*28).to(DEVICE)
        label = label.to(DEVICE)
        
        encoded, classified, decoded = autoencoder(x)
        
        loss = ae_criterion(decoded, y)
        if step == 1:
            loss_history.append(loss.data)
        
        ae_optimizer.zero_grad()
        loss.backward()
        ae_optimizer.step()
        
def classifier_train(autoencoder, train_loader):
    autoencoder.train()
    for step, (x, label) in enumerate(train_loader):
        x = x.view(-1, 28*28).to(DEVICE)
        y = x.view(-1, 28*28).to(DEVICE)
        label = label.to(DEVICE)
        
        encoded, classified, decoded = autoencoder(x)
        
        loss = a

In [7]:
for epoch in range(1, EPOCH+1):
    ae_train(autoencoder, train_loader)
    # ae_train 완료
    
    test_x = view_data.to(DEVICE)
    
    _, classified_data, decoded_data = autoencoder(test_x)
    
    f, a = plt.subplots(2, 10, figsize=(10, 2))
    print("[Epoch {}]".format(epoch))
    for i in range(10):
        img = np.reshape(view_data.data.numpy()[i],(28,28))
        a[0][i].imshow(img, cmap='gray')
        a[0][i].set_xticks(()); a[0][i].set_yticks(())
        
    for i in range(10):
        img = np.reshape(decoded_data.to("cpu").data.numpy()[i], (28, 28))
        a[1][i].imshow(img, cmap='gray')
        a[1][i].set_xticks(()); a[1][i].set_yticks(())
        
    plt.show()

NameError: name 'loss_history' is not defined

In [ ]:
with torch.no_grad():
    X_test = test_loader.test_data.view(-1, 28*28).float().to(DEVICE)
    Y_test = test_loader.test_labels.to(DEVICE)
    
    _, prediction, _ = autoencoder(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    r = random.randint(0, len(test_loader) - 1)
    X_single_data = test_loader.test_data[r:r+1].view(-1, 28*28).float().to(DEVICE)
    Y_single_data = test_loader.test_labels[r:r+1].to(DEVICE)
    
    print("Label: ", Y_single_data.item())
    _, single_prediction, _ = autoencoder(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())
    
    plt.imshow(test_loader.test_data[r:r+1].view(28, 28), cmap='Greys', interpolation='nearest')
    plt.show()